# CSC/XMM validation catalogs, XMM/CSC hostless catalog

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

import matplotlib.pyplot as plt
# plt.style.use('seaborn')

import seaborn as sns

import json

from scripts.utils import set_mpl, data_path
from scripts.cross_match_scripts import cross_match_data_frames, fits_to_pandas, add_separation_columns
set_mpl()

%matplotlib inline
from astropy.table import Table

matplotlib settings set


matplotlib settings set
matplotlib settings set


In [3]:
def csc_error_converter(df: pd.DataFrame,
                        r0_colname='err_ellipse_r0',
                        r1_colname='err_ellipse_r1') -> pd.DataFrame:
    """
    The function converts default radii `r0_colname`
    and `r1_colname` to the one-sigma error.

    Args:
        df (pd.DataFrame): DataFrame with `r0_colname` and
        `r1_colname` columns.
        
        r0_colname (str): major radius of the 95% confidence
        level position error ellipse.
        Defaults to 'err_ellipse_r0'.
        
        r1_colname (str): minor radius of the 95% confidence
        level position error ellipse.
        Defaults to 'err_ellipse_r1'.

    Returns:
        pd.DataFrame: one-sigma error in arcseconds.
    """

    # Conversion coefficient
    csc_sigma_coeff = np.sqrt(-(1 / (2 * np.log(1 - .95))))

    err_r1 = df[r0_colname]
    err_r2 = df[r1_colname]
    # Effective error
    csc_err_eff = np.sqrt(err_r1 ** 2 + err_r2 ** 2)

    csc_1sigma = csc_sigma_coeff * csc_err_eff

    return csc_1sigma


def vot2pd_csc(csc_cat_path: str,
               colnames: list,
               save_coords: bool = False, 
               radec_fits_name: str = 'cscresults_name_radec') -> pd.DataFrame:
    """
    The function converts the votable file to pandas DataFrame.

    Optionally saves the separate file with the coordinates and names of
    the CSC sources to the FITS file.

    Args:
        csc_cat_path (str): the path to the votable file.
        colnames (list): column names of the votable file.
        save_coords (bool): if True, saves the separate file with the coordinates
        and names of the CSC sources to the FITS file.
        radec_fits_name (str): name of the FITS file with
        the coordinates and names of the CSC sources to be saved.

    Returns:
        pd.DataFrame: converted catalogue.
    """
    
    vot_table = Table.read(csc_cat_path, format='votable')
    
    df = vot_table.to_pandas()
    df.columns = colnames

    df = df.assign(flux_csc_05_2 = lambda x: x.flux_aper_s + x.flux_aper_m)

    one_sigma_errors = csc_error_converter(df)

    df = df.assign(radec_err_csc = one_sigma_errors)

    df['r_98_csc'] = tsource_r(sigma=df['radec_err_csc'], t_thresh=.02)

    if save_coords:

        df_init_coords = df[['name', 'ra', 'dec']]
        Table.from_pandas(df_init_coords).write(f'data/{radec_fits_name}.fits', format='fits')

    return df


def tsource_r(sigma: float, t_thresh: float) -> float:
    """
    Calculates the radius of circle which contains the probability `t_thresh`
    NOT to find a counterpart for a source with localization error `sigma`.

    Args:
        sigma (float): localization error in arcsec.
        t_thresh (float): probability to NOT find a counterpart.

    Returns:
        float: radius of circle in arcsec.
    """
    
    rsearch = sigma * np.sqrt(-2 * np.log(t_thresh))
    
    return rsearch


def fsource_r(rho: float, f_thresh: float) -> float:
    """
    The function calculates the radius which corresponds to the probability `f_thresh`
    to FIND one or more false sources. 

    Args:
        rho (float): the density of the false sources (in arcmin^{-2}).
        f_thresh (float): probability to FIND one or more false sources.

    Returns:
        float: radius (in arcsec).
    """

    rho = rho / 3600 # arcmin^{-2} -> arcsec^{-2}
    pf_r = np.sqrt(-1 / (rho * np.pi) * np.log(1 - f_thresh))
    
    return pf_r


def poserr2sigma_coeff(conf_level: float) -> float:
    """
    Calculates convertion coefficient to go from
    positional error to sigma given confidence level.

    For details see:
    https://www.notion.so/Theory-ca6e7795b40c43b4ba6d96bc59727efa#b4d9fc11ff8243a3834e9eeba08c2273
    """
    coeff = (-2 * np.log(1 - conf_level)) ** -0.5
    
    return coeff


def pos_r(sigma: float, conf_level: float) -> float:
    """
    Calculates the radius of circle which contains the probability `conf_level`
    of finding (opposite to tsource_r()) a counterpart for a source with
    localization error `sigma`.

    For details see:
    https://www.notion.so/Theory-ca6e7795b40c43b4ba6d96bc59727efa#0ba88df64d2d4d9583f93d63dbe1b927

    Args:
        sigma (float): localization error in arcsec.
        conf_level (float): probability of finding a counterpart.

    Returns:
        float: radius of circle in arcsec.
    """
    
    r_pos = sigma * np.sqrt(-2 * np.log(1 - conf_level))

    return r_pos


# def only_reliable_xmm(df: pd.DataFrame) -> pd.DataFrame:
#     """
#     Filters out unreliable XMM sources.
#     """
#     reliable_df = df[
#                     ((df['xmm_SC_SUM_FLAG'] == 0) |
#                     (df['xmm_SC_SUM_FLAG'] == 1)) &
#                     (df['xmm_SC_DET_ML'] > 10) &
#                     ~(df['xmm_SC_VAR_FLAG'] == True) &
#                     (df['xmm_SC_EXTENT'] == 0) &
#                     (df['xmm_CONFUSED'] == False)
#                     ]

#     return reliable_df


def only_reliable_xmm(df: pd.DataFrame) -> pd.DataFrame:
    """
    Filters out unreliable XMM sources.
    """
    reliable_df = df[
                    ((df['SC_SUM_FLAG'] == 0) |
                    (df['SC_SUM_FLAG'] == 1)) &
                    (df['SC_DET_ML'] > 10) &
                    ~(df['SC_VAR_FLAG'] == True) &
                    (df['SC_EXTENT'] == 0) &
                    (df['CONFUSED'] == False)
                    ]

    return reliable_df




def xray_filtration(df: pd.DataFrame,
                    DL_thresh: float = 6,
                    EL_thresh: float = 6,
                    verbouse=True) -> pd.DataFrame:
    """
    Filters X-ray sources.
    TODO: remake processing of duplicates
    """
    
    if verbouse:
        print(f'DET_LIKE_0 > {DL_thresh}')
        print(f'EXT_LIKE < {EL_thresh}')
        print()

        print(f'Before X-ray source filters: {len(df)}')

    df = df[(df['DET_LIKE_0'] > DL_thresh)&
            (df['EXT_LIKE'] < EL_thresh)]

    if verbouse:
        print(f'After X-ray source filters: {len(df)}')
        print()


    # Manually get rid of faint sources in duplicated pairs
    df = df[~((df['srcname_fin']=='SRGe J104659.3+573056')&(df['DET_LIKE_0'] < 20))]
    df = df[~((df['srcname_fin'] == 'SRGe J104700.7+574558')&(df['DET_LIKE_0'] < 20))]
    print('Weak ERO duplicates removed (temporary measure)')
    print()

    return df



srg_names = {
            'id_src_name': 'srcname_fin',  # Индексы рентгеновских источников
            'x_ra_name': 'RA_fin',  # Координаты рентгеновских источников
            'x_dec_name': 'DEC_fin',
            'dl_name': 'DET_LIKE_0',  # Detection Likelihood
            'x_flux_name': 'flux_05-20',
            'ext_name': 'EXT_LIKE',  # Протяженность рентгеновских источников
            'ls_ra_name': 'ra',  # Координаты источников DESI
            'ls_dec_name': 'dec',
            'r_98_name': 'pos_r98',  # Позиционная ошибка
            'sigma_2d_name': 'pos_sigma_2d'
            }


***

In [4]:
# Каталог ERO (9500)
ero_df = pd.read_pickle(data_path+'ERO_lhpv_03_23_sd01_a15_g14.pkl')
ero_df = xray_filtration(ero_df, DL_thresh=6, EL_thresh=6)
ero_df.sample(5)

DET_LIKE_0 > 6
EXT_LIKE < 6

Before X-ray source filters: 9215
After X-ray source filters: 9215

Weak ERO duplicates removed (temporary measure)



,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH
4834,SRGe J104638.4+545721,161.660158,54.955754,153.446322,54.139492,6.277092,33.693470,7.108890e-15,1.347584e-15,49.816036,9.443286,6401.988281,0.0,0.0,0.0,1112,897,161.662166,54.954464,2.675648,53.710861,0.000003,6.268413e+08,6.270701e+08,0.0,0.0,0.007781,0.001475,30.307577,161.660158,54.955754,-1,-1.000000,0,-1,NaN,-1.0,-1,SDSS J104638.10+545723.7,1.619269,QSO,10.403754,1,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,74749845156,2.675648,2.244103,3.387681,5.492996,137.961010,42.787545,5.470568e-15,9.013231e+19
367,SRGe J105845.0+574733,164.687457,57.792400,147.965977,53.511559,3.113070,351.871552,5.080259e-14,4.136381e-15,185.683395,15.118466,3339.140869,0.0,0.0,0.0,507,425,164.689466,57.791110,1.099144,80.356819,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.055608,0.004528,16.019707,164.687457,57.792400,857219813801847808,5.377824,1,0,19.284580,-1.0,0,SDSS J105845.00+574732.3,0.963000,QSO,5.378150,1,0,0,1,15.990000,14.761,12.134,21.4,20.1,3.3,1,1,1,1237658302743117984,7996261097001996288,5.419111,63446313896,1.099144,1.112944,1.680091,2.724204,137.825724,46.080372,3.909457e-14,5.995505e+19
1787,SRGe J102827.0+553900,157.112348,55.649848,155.008759,51.630967,8.148797,25.404057,1.689710e-14,3.864665e-15,30.861286,7.058523,1668.589233,0.0,0.0,0.0,4463,3592,157.114356,55.648559,3.545043,271.603973,0.000014,6.268413e+08,6.270701e+08,0.0,0.0,0.018495,0.004230,8.151438,157.112348,55.649848,853739859499428736,9.634991,1,0,20.915537,-1.0,0,SDSS J102826.31+553853.6,0.712000,QSO,9.666027,1,0,0,1,16.419001,15.455,12.261,17.0,12.1,1.3,1,1,1,1237655106760671443,7539267226362466304,9.636395,71905372868,3.545043,2.913250,4.397820,7.130899,134.458289,42.081347,1.300297e-14,6.221632e+19
7146,SRGe J104939.6+581912,162.414987,58.319961,148.713558,52.278531,13.179267,6.962558,4.634399e-15,1.554120e-15,21.753166,7.294801,4288.214844,0.0,0.0,0.0,6932,5820,162.416996,58.318671,5.835861,98.940865,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.005073,0.001701,18.930315,162.414987,58.319961,-1,-1.000000,0,-1,NaN,-1.0,-1,NaN,NaN,NaN,-1.000000,0,0,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,0,0,-1,0,0,NaN,61437709402,5.835861,4.711678,7.112711,11.532993,135.963119,45.861461,3.566351e-15,8.097649e+19
2014,SRGe J104645.6+564744,161.690064,56.795548,151.023759,52.992424,4.996670,71.531296,1.547499e-14,2.305538e-15,69.072014,10.290683,4077.738037,0.0,0.0,0.0,555,461,161.692073,56.794259,2.065857,43.407909,0.000007,6.268413e+08,6.270701e+08,0.0,0.0,0.016939,0.002524,16.973787,161.690064,56.795548,854059954822456448,5.868726,1,1,11.078521,NaN,0,TYC 3826-662-1,-0.000026,Star,5.553877,2,0,1,0,9.697000,9.730,9.702,50.2,54.1,27.4,1,1,0,1237658302205460497,0,5.650011,67322444474,2.065857,1.786343,2.696650,4.372517,136.656697,44.366311,1.190861e-14,6.536069e+19


In [5]:
# desi_lh.gz_pkl number of sources and area
false_dens_arcmin = 2418574 / (41.729 * 3600)
print(false_dens_arcmin)
false_dens_arcsec = false_dens_arcmin / 3600

print(f'{ false_dens_arcsec:.3f}')

FALSE_PROB = .03

r_false = fsource_r(false_dens_arcmin, FALSE_PROB)

print(f'Радиус для {FALSE_PROB:.0%} вероятности найти ложный: {r_false:.2f}"')

desi = pd.read_pickle(data_path+'desi_lh.gz_pkl', compression='gzip')
desi.sample(5)

16.099741453452303
0.004
Радиус для 3% вероятности найти ложный: 1.47"


,release,objid,brickid,ra,dec,flux_g,flux_r,flux_z,flux_w1,flux_w2,flux_w3,flux_w4,flux_ivar_g,flux_ivar_r,flux_ivar_z,flux_ivar_w1,flux_ivar_w2,flux_ivar_w3,flux_ivar_w4,dered_mag_g,dered_mag_r,dered_mag_z,dered_mag_w1,dered_mag_w2,dered_mag_w3,dered_mag_w4,snr_g,snr_r,snr_z,snr_w1,snr_w2,snr_w3,snr_w4,type,parallax,parallax_ivar,pmra,pmra_ivar,pmdec,pmdec_ivar,ref_cat,ref_id,mjd_max,mjd_min,iso_max,iso_min,desi_id
2066786,9011,2113,609934,155.729290,57.520509,0.094688,0.248902,0.693995,0.966940,1.320155,31.553970,214.58723,600.73740,239.18680,175.922200,4.573809,1.103873,0.001511,0.000018,24.984282,23.959425,22.868360,22.532210,22.195800,18.751802,16.670778,2.320809,3.849433,9.204844,2.067944,1.387026,1.226375,0.901859,PSF,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.391325,57491.210880,2018-03-17 09:23:30.460,2016-04-13 05:03:40.000,9011_609934_2113
1949420,9011,2517,600236,160.555369,54.463107,0.864469,2.258822,7.657409,30.700716,16.983614,104.545654,367.10153,396.27872,162.98570,36.781670,3.261081,0.963000,0.001174,0.000015,22.623713,21.592113,20.276829,18.780159,19.423716,17.451477,16.087936,17.208763,28.837444,46.440575,55.440777,16.666460,3.581481,1.417889,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57897.268044,57510.175311,2017-05-24 06:25:59.000,2016-05-02 04:12:26.910,9011_600236_2517
2022364,9011,1019,612242,155.083731,58.136136,0.315813,0.592369,0.677757,-0.171999,-1.248061,29.901901,261.95602,560.80597,174.72412,64.566100,4.465908,1.075321,0.001516,0.000018,23.720818,23.047903,22.910782,NaN,NaN,18.810524,16.454342,7.478881,7.830125,5.445984,-0.363481,-1.294210,1.164319,1.096949,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.386955,57842.273037,2018-03-17 09:17:12.879,2017-03-30 06:33:10.361,9011_612242_1019
1172079,9011,2410,612244,156.164934,58.263015,0.299728,0.656343,1.110985,0.360005,0.288226,-18.496490,136.17314,1113.59990,336.62674,100.426674,4.459905,1.082484,0.001421,0.000017,23.776287,22.935690,22.373713,23.607403,23.849546,NaN,17.164680,10.002129,12.042173,11.133524,0.760276,0.299877,-0.697237,0.568812,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,58194.391325,57820.343785,2018-03-17 09:23:30.460,2017-03-08 08:15:03.000,9011_612244_2410
827817,9011,1114,612264,165.482072,58.156221,0.451180,0.855542,2.847807,7.530406,8.016585,-26.515759,-182.51134,328.33710,112.22603,36.794865,4.273222,1.065898,0.001008,0.000012,23.339808,22.653017,21.354560,20.306562,20.239172,NaN,NaN,8.175419,9.063334,17.274450,15.566684,8.276511,-0.841689,-0.644155,REX,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0,57920.259630,57545.197768,2017-06-16 06:13:52.000,2016-06-06 04:44:47.170,9011_612264_1114


# CSC

In [6]:
# CSC table conversion
csc_columns = ['name', 'ra', 'dec', 'err_ellipse_r0', 'err_ellipse_r1', 'err_ellipse_ang',
               'significance', 'likelihood', 'likelihood_class', 'conf_flag', 'dither_warning_flag',
               'extent_flag', 'pileup_flag', 'sat_src_flag', 'streak_src_flag', 'var_flag',
               'flux_aper_s', 'flux_aper_lolim_s', 'flux_aper_hilim_s', 'flux_aper_m',
               'flux_aper_lolim_m', 'flux_aper_hilim_m']

# Filtration
# data/cscresults.vot obtained via CSCview software
csc_init_df = vot2pd_csc(csc_cat_path=data_path+'cscresults.vot', colnames=csc_columns)

# CSC catalogue filtering
csc_df = csc_init_df[(csc_init_df['conf_flag'] == False) &
                              (csc_init_df['extent_flag'] == False) &
                              (csc_init_df['sat_src_flag'] == False) &
                              (csc_init_df['streak_src_flag'] == False) &
                              (csc_init_df['pileup_flag'] == False) &
                              (csc_init_df['dither_warning_flag'] == False)]


csc_df = csc_df[(csc_df['likelihood'] > 10) &
                                  (csc_df['likelihood'] < 10 ** 10)]


csc_df = csc_df.assign(flux_05_2 = lambda x: x.flux_aper_s + x.flux_aper_m)

# Ошибки на поток 0.5-2 кэВ
s_up = csc_df.flux_aper_hilim_s - csc_df.flux_aper_s
s_down = csc_df.flux_aper_s - csc_df.flux_aper_lolim_s
s_err = np.sqrt(s_up ** 2 + s_down ** 2)
m_up = csc_df.flux_aper_hilim_m - csc_df.flux_aper_m
m_down = csc_df.flux_aper_m - csc_df.flux_aper_lolim_m
m_err = np.sqrt(m_up ** 2 + m_down ** 2)

sm_err = np.sqrt(s_err ** 2 + m_err ** 2).values
csc_df['flux_aper_sm_err'] = sm_err


print(f'CSC Sources: {csc_df.shape[0]}')
csc_df.sample(5)

CSC Sources: 267265


,name,ra,dec,err_ellipse_r0,err_ellipse_r1,err_ellipse_ang,significance,likelihood,likelihood_class,conf_flag,dither_warning_flag,extent_flag,pileup_flag,sat_src_flag,streak_src_flag,var_flag,flux_aper_s,flux_aper_lolim_s,flux_aper_hilim_s,flux_aper_m,flux_aper_lolim_m,flux_aper_hilim_m,flux_csc_05_2,radec_err_csc,r_98_csc,flux_05_2,flux_aper_sm_err
198786,2CXO J145719.6-211942,224.331865,-21.328391,1.096595,0.851184,151.915634,2.756757,15.062701,MARGINAL,False,False,False,False,False,False,False,1.084536e-15,6.691819e-16,1.499891e-15,2.306568e-16,3.008566e-17,4.211993e-16,1.315193e-15,0.567124,1.586332,1.315193e-15,6.492870e-16
8282,2CXO J004246.8+411821,10.695366,41.305929,0.928091,0.803719,133.288159,2.051282,23.196195,TRUE,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.501575,1.402980,NaN,NaN
292831,2CXO J205143.4-524333,312.931099,-52.725836,1.467070,1.197331,32.380775,2.102564,14.684169,TRUE,False,False,False,False,False,False,False,1.245103e-15,2.873315e-16,2.202875e-15,NaN,NaN,NaN,NaN,0.773628,2.163954,NaN,NaN
162006,2CXO J124619.3+304551,191.580747,30.764325,2.032575,2.032579,0.000000,2.378378,12.718997,MARGINAL,False,False,False,False,False,False,False,2.854373e-16,7.135932e-17,4.995152e-16,8.210215e-16,3.889049e-16,1.231532e-15,1.106459e-15,1.174344,3.284817,1.106459e-15,6.685077e-16
315835,2CXO J235233.0-465431,358.137795,-46.908744,0.780426,0.747786,170.378472,3.875000,68.570077,TRUE,False,False,False,False,False,False,False,1.457755e-16,2.650463e-17,2.584201e-16,6.751598e-16,3.989581e-16,9.360170e-16,8.209352e-16,0.441572,1.235142,8.209352e-16,4.138215e-16


## CSC x ERO cross-match

In [7]:
ero_csc = cross_match_data_frames(ero_df, csc_df,
 'RA_fin', 'DEC_fin', 'ra', 'dec', match_radius = 30, df_prefix = 'csc')
ero_csc = ero_csc.query("csc_n_near==1 & csc_n_matches==1")
print('cross-matches: only one csc within 30'' and it is unique', ero_csc.shape[0])
assert len(ero_csc) == ero_csc.csc_name.nunique()

ero_csc.sample(5)

cross-match radius 30 arcsec
total matches: 739 out of 9215 x 267265
	 total unique pairs: 692
	 total non-unique pairs (duplicates in df2): 47
cross-matches: only one csc within 30 and it is unique 566


,srcname_fin,RA_fin,DEC_fin,GLON,GLAT,pos_r98,DET_LIKE_0,ML_FLUX_0,ML_FLUX_ERR_0,ML_CTS_0,ML_CTS_ERR_0,ML_EXP_1,EXT,EXT_LIKE,EXT_ERR,ID_SRC,ID_CLUSTER,RA,DEC,RADEC_ERR,DIST_NN,SRCDENS,TSTART,TSTOP,EXT_LOWERR,EXT_UPERR,ML_RATE_0,ML_RATE_ERR_0,ML_BKG_0,RA_corr,DEC_corr,g_id,g_d2d,g_nsrc,g_s,g_gmag,g_maxLx,g_b,s_id,s_z,s_otype,s_d2d,s_nsrc,flag_xray,flag_radio,flag_agn_wise,w1,w2,w3,w1snr,w2snr,w3snr,w_nsrc,sdss_nsrc,sdss_p,sdss_id,sdss_sp,sdss_d2d,hpidx,RADEC_ERR_fin,pos_sigma_2d,pos_r68,pos_r95,ELON,ELAT,flux_05-20,NH,csc_name,csc_ra,csc_dec,csc_err_ellipse_r0,csc_err_ellipse_r1,csc_err_ellipse_ang,csc_significance,csc_likelihood,csc_likelihood_class,csc_conf_flag,csc_dither_warning_flag,csc_extent_flag,csc_pileup_flag,csc_sat_src_flag,csc_streak_src_flag,csc_var_flag,csc_flux_aper_s,csc_flux_aper_lolim_s,csc_flux_aper_hilim_s,csc_flux_aper_m,csc_flux_aper_lolim_m,csc_flux_aper_hilim_m,csc_flux_csc_05_2,csc_radec_err_csc,csc_r_98_csc,csc_flux_05_2,csc_flux_aper_sm_err,csc_sep,csc_n_near,csc_n_matches
542,SRGe J105032.3+564802,162.634698,56.800520,150.454629,53.381393,8.156309,15.783928,5.440227e-15,1.493614e-15,25.097309,6.890465,4214.612305,0.0,0.0,0.0,4985,4056,162.636707,56.799231,3.548501,48.013638,0.000009,6.268413e+08,6.270701e+08,0.0,0.0,0.005955,0.001635,18.993195,162.634698,56.800520,851138410692641024,6.102985,1,0,20.943577,-1.0,0,SDSS J105032.58+564803.0,1.960760,QSO,6.102814,1,1,0,0,17.341000,17.313,12.308,8.7,2.4,2.5,1,1,1,1237655106762113220,9207753598804709376,6.106220,67302633103,3.548501,2.915936,4.401874,7.137472,137.267627,44.645162,4.186467e-15,6.959493e+19,2CXO J105032.1+564804,162.634075,56.801189,9.987116,9.987116,0.000000,2.810811,19.494125,TRUE,False,False,False,False,False,False,False,0.000000e+00,0.000000e+00,2.601999e-15,7.753525e-15,4.245978e-15,1.126107e-14,7.753525e-15,5.770170,16.140029,7.753525e-15,5.601444e-15,2.703348,1,1
622,SRGe J103105.9+573747,157.774385,57.629820,152.074837,50.808386,8.188158,10.766065,4.369473e-15,1.332926e-15,20.397018,6.222196,4264.666992,0.0,0.0,0.0,7796,6659,157.776394,57.628530,3.563162,166.531876,0.000009,6.268413e+08,6.270701e+08,0.0,0.0,0.004783,0.001459,19.348345,157.774385,57.629820,-1,-1.000000,0,-1,NaN,-1.0,-1,CLASXS 4,NaN,X,6.104096,1,1,0,0,16.448999,15.984,12.784,16.8,7.9,-0.1,1,1,0,1237655108908614518,0,8.183359,64071216249,3.563162,2.927322,4.419063,7.165343,133.527168,43.975964,3.362480e-15,6.423606e+19,2CXO J103105.7+573749,157.773870,57.630436,2.311181,1.299491,86.663187,3.352941,29.913404,TRUE,False,False,False,False,False,False,False,1.033129e-15,4.893771e-16,1.576882e-15,1.083824e-15,6.021244e-16,1.541439e-15,2.116953e-15,1.083224,3.029941,2.116953e-15,1.016257e-15,2.430322,1,1
24,SRGe J103440.0+574355,158.666611,57.732003,151.493369,51.130725,2.868944,612.382446,6.337805e-14,4.004150e-15,300.247955,18.969309,4328.014160,0.0,0.0,0.0,237,197,158.668620,57.730713,0.956876,116.449554,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.069373,0.004383,19.187504,158.666611,57.732003,854824282906636544,6.917654,1,0,20.824614,-1.0,0,2XMM J103439.8+574355,0.451227,QSO,6.893005,1,1,0,1,15.264000,14.436,11.543,30.7,24.2,7.0,1,2,0,1237655108908810471,7980580411771473920,7.042223,63678056154,0.956876,1.025667,1.548339,2.510573,134.023771,44.309107,4.877189e-14,5.741597e+19,2CXO J103439.8+574355,158.666206,57.732097,0.727052,0.723881,37.116559,14.781488,1306.861205,TRUE,False,False,False,False,False,False,True,8.740704e-15,7.917488e-15,9.563921e-15,9.754971e-15,8.983128e-15,1.048141e-14,1.849568e-14,0.419148,1.172418,1.849568e-14,1.574430e-15,0.848898,1,1
117,SRGe J105414.9+571241,163.562132,57.211433,149.374494,53.480216,4.754669,123.576431,2.149584e-14,2.562719e-15,96.194374,11.468227,4088.300781,0.0,0.0,0.0,1362,1079,163.564140,57.210144,1.948100,92.100815,0.000006,6.268413e+08,6.270701e+08,0.0,0.0,0.023529,0.002805,16.813267,163.562132,57.211433,851175038174127616,7.512929,1,0,19.214256,-1.0,0,[VV2006] J105414.7+571241,1.796120,QSO,7.509573,1,1,1,1,16.343000,14.989

## CSC x DESI cross-match

In [8]:
csc_desi = cross_match_data_frames(ero_csc, desi, colname_ra1 = 'csc_ra',colname_dec1 = 'csc_dec' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')

cross-match radius 15 arcsec
total matches: 2543 out of 566 x 2418574
	 total unique pairs: 2543
	 total non-unique pairs (duplicates in df2): 0


In [9]:
csc_desi_closest = (csc_desi
    .loc[csc_desi.groupby('csc_name')['desi_sep'].idxmin()]
    )


csc_hostless = (csc_desi_closest[
    csc_desi_closest['desi_sep'] > 2 * csc_desi_closest['csc_r_98_csc']
    ])
csc_hostless = csc_hostless.query('csc_sep<20')
csc_hostless['csc_ero_flux_ratio'] = csc_hostless['csc_flux_csc_05_2'] / csc_hostless['flux_05-20']

print('hostless csc near chandra sources', csc_hostless.shape[0])
csc_hostless = csc_hostless[['srcname_fin',	'RA_fin',	'DEC_fin', 'pos_r98', 'flux_05-20', 'csc_name', 'csc_ra',	'csc_dec', 'csc_r_98_csc', 'csc_flux_05_2', 'csc_sep', 'desi_desi_id', 'desi_ra', 'desi_dec', 'desi_sep', 'csc_ero_flux_ratio']]

csc_hostless.rename(columns = {'csc_sep':'ero_csc_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

csc_hostless = add_separation_columns(csc_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert csc_hostless.srcname_fin.nunique()==csc_hostless.csc_name.nunique()
assert csc_hostless.srcname_fin.nunique()==csc_hostless.shape[0]
csc_hostless.sample(7)

hostless csc near chandra sources 29


,srcname_fin,RA_fin,DEC_fin,pos_r98,flux_05-20,csc_name,csc_ra,csc_dec,csc_r_98_csc,csc_flux_05_2,ero_csc_sep,desi_desi_id,desi_ra,desi_dec,desi_sep_minimal,csc_ero_flux_ratio,desi_ero_sep
2528,SRGe J104341.0+590023,160.920944,59.006447,10.333953,2.463644e-15,2CXO J104341.2+590021,160.921860,59.005939,1.845178,1.203987e-15,2.495724,9011_614515_1592,160.927624,59.007158,11.550977,0.488702,12.643794
2478,SRGe J105323.6+573829,163.348168,57.641255,9.384750,2.619025e-15,2CXO J105324.2+573828,163.350964,57.641244,1.679285,NaN,5.388710,9011_610726_1110,163.348859,57.640138,5.684306,NaN,4.234390
925,SRGe J103239.4+574033,158.163965,57.675913,6.556224,8.782485e-15,2CXO J103239.3+574035,158.164092,57.676566,1.199912,8.115216e-15,2.364492,9011_610715_599,158.169132,57.676603,9.702210,0.924023,10.251895
2085,SRGe J103520.2+573355,158.834253,57.565203,8.727764,3.591921e-15,2CXO J103520.9+573349,158.837140,57.563679,2.384348,4.241985e-15,7.821427,9011_609941_914,158.835675,57.562204,6.015550,1.180980,11.139738
2259,SRGe J104646.4+585616,161.693470,58.937778,9.715281,3.165194e-15,2CXO J104647.0+585617,161.696030,58.938273,1.515592,8.078599e-15,5.077537,9011_614517_243,161.692534,58.936381,9.410265,2.552323,5.320833
1495,SRGe J104854.4+573926,162.226706,57.657139,8.499000,5.270004e-15,2CXO J104853.5+573920,162.222929,57.655774,2.009572,7.151569e-15,8.777265,9011_610723_3980,162.217314,57.657324,12.169596,1.357033,18.101054
2402,SRGe J103402.4+574359,158.510143,57.732976,10.365024,2.877164e-15,2CXO J103401.9+574356,158.507957,57.732339,1.283418,4.631479e-16,4.785909,9011_610715_3300,158.510277,57.732739,4.686013,0.160974,0.889982


***

# 4XMM DR10

## XMM x EROSITA (done in topcat)

In [10]:
ero_xmm = pd.read_csv(data_path+'xmm_allsky_full_ero_slim_point_30sec.csv')
ero_xmm = only_reliable_xmm(ero_xmm) #TODO CHECK IF XMM IS RELIABLE

print(f'Reliable XMM Sources within 30 arcsec from eROSITA: {ero_xmm.shape[0]}')

# Поток 0.5-2 кэВ
ero_xmm = ero_xmm.assign(flux_05_2 = lambda x: x.SC_EP_2_FLUX + x.SC_EP_3_FLUX)
# Ошибки на поток 0.5-2 кэВ
ero_xmm = ero_xmm.assign(flux_05_2_err = lambda x: np.sqrt(x.SC_EP_2_FLUX_ERR ** 2 + x.SC_EP_3_FLUX_ERR ** 2))

xmm_err = ero_xmm['SC_POSERR']
xmm_sigma_coeff = poserr2sigma_coeff(.63)
xmm_sigma = xmm_sigma_coeff * xmm_err
ero_xmm.insert(11, 'sigma', xmm_sigma)
# pos_r98
xmm_r98 = pos_r(xmm_sigma, .98)
ero_xmm.insert(11, 'xmm_pos_r98', xmm_r98)


ero_xmm['xmm_ero_flux_ratio'] = ero_xmm['flux_05_2']\
                                                / ero_xmm['flux_05-20']



ero_xmm['GroupSize'] = ero_xmm['GroupSize'].fillna(1)
ero_xmm = ero_xmm[ero_xmm['GroupSize']==1]

print(f'XMM Sources after filters (only one within 30 arcsec): {ero_xmm.shape[0]}')



ero_xmm.sample(10)

Reliable XMM Sources within 30 arcsec from eROSITA: 844
XMM Sources after filters (only one within 30 arcsec): 740


,srcname_fin,RA_fin,DEC_fin,flux_05-20,pos_sigma_2d,pos_r98,DET_LIKE_0,EXT_LIKE,pos_r98_corr,SRCID,IAUNAME,xmm_pos_r98,sigma,SC_RA,SC_DEC,SC_POSERR,SC_DET_ML,SC_EP_1_FLUX,SC_EP_1_FLUX_ERR,SC_EP_2_FLUX,SC_EP_2_FLUX_ERR,SC_EP_3_FLUX,SC_EP_3_FLUX_ERR,SC_EP_4_FLUX,SC_EP_4_FLUX_ERR,SC_EP_5_FLUX,SC_EP_5_FLUX_ERR,SC_EP_8_FLUX,SC_EP_8_FLUX_ERR,SC_EP_9_FLUX,SC_EP_9_FLUX_ERR,SC_HR1,SC_HR1_ERR,SC_HR2,SC_HR2_ERR,SC_HR3,SC_HR3_ERR,SC_HR4,SC_HR4_ERR,SC_EXTENT,SC_EXT_ERR,SC_EXT_ML,SC_CHI2PROB,SC_FVAR,SC_FVARERR,SC_VAR_FLAG,SC_SUM_FLAG,SC_EP_8_FMIN,SC_EP_8_FMIN_ERR,SC_EP_8_FMAX,SC_EP_8_FMAX_ERR,MJD_FIRST,MJD_LAST,N_DETECTIONS,CONFUSED,WEBPAGE_URL,GroupID,GroupSize,Separation,flux_05_2,flux_05_2_err,xmm_ero_flux_ratio
61,SRGe J104233.9+584055,160.641352,58.682004,3.676054e-14,1.035681,2.896955,427.100000,0.0,5.000000,205562113010016,4XMM J104234.1+584054,3.262712,1.166442,160.642229,58.681777,1.644850,53.9033,6.900630e-15,1.455650e-15,3.446310e-15,1.164370e-15,1.110280e-15,8.778650e-16,3.444900e-15,2.520520e-15,0.000000e+00,9.681610e-15,1.785960e-14,1.044370e-14,8.707710e-15,2.524890e-15,-0.389038,0.169136,-0.165118,0.255188,-0.520620,0.319654,-1.000000,1.695730,0.0,NaN,-0.273174,NaN,NaN,NaN,NaN,0,1.785960e-14,1.044370e-14,1.785960e-14,1.044370e-14,54790.834549,54790.915961,1,False,http://xmm-catalog.irap.omp.eu/source/20556211...,NaN,1.0,1.831466,4.556590e-15,1.458220e-15,0.123953
674,SRGe J105236.2+571606,163.150849,57.268201,5.026697e-15,3.099569,8.669957,15.772934,0.0,9.536953,201237001010052,4XMM J105236.3+571602,0.756750,0.270543,163.151389,57.267263,0.381505,183.1840,9.110230e-16,9.969620e-17,2.103710e-15,1.352480e-16,1.936240e-15,1.492260e-16,2.774110e-15,3.561350e-16,6.520920e-15,2.208170e-15,1.662390e-14,2.294500e-15,9.431150e-15,3.897780e-16,0.324620,0.046954,0.012263,0.041498,-0.320105,0.051209,0.377656,0.088994,0.0,NaN,-0.636822,0.777169,NaN,NaN,False,1,9.991320e-15,1.019760e-14,5.112300e-14,2.045290e-14,51661.149132,52615.192419,12,False,http://xmm-catalog.irap.omp.eu/source/20123700...,NaN,1.0,3.534665,4.039950e-15,2.013962e-16,0.803699
743,SRGe J104354.9+592856,160.978829,59.482102,4.428224e-15,3.595987,10.058513,13.518688,0.0,11.064364,205562129010024,4XMM J104354.6+592840,3.669031,1.311703,160.977692,59.477809,1.849690,19.8810,5.408680e-16,5.459440e-16,2.080980e-15,9.195100e-16,8.059180e-16,6.305970e-16,4.035710e-15,2.564080e-15,6.706310e-15,8.572670e-15,1.840000e-14,9.187150e-15,1.013600e-14,2.477200e-15,0.318593,0.211726,-0.649459,0.158959,-0.098338,0.295424,-0.271217,0.453038,0.0,NaN,-0.776503,NaN,NaN,NaN,NaN,0,1.840000e-14,9.187150e-15,1.840000e-14,9.187150e-15,54797.077187,54797.181748,1,False,http://xmm-catalog.irap.omp.eu/source/20556212...,NaN,1.0,15.594562,2.886898e-15,1.114967e-15,0.651931
685,SRGe J104654.5+585802,161.726940,58.967249,4.975153e-15,2.956846,8.270740,18.992867,0.0,9.097814,205541201015078,4XMM J104654.4+585801,2.022312,0.722990,161.726864,58.967196,1.019520,40.2578,1.700210e-15,3.018380e-16,1.351890e-15,2.849830e-16,8.436590e-16,3.012000e-16,1.308390e-15,6.232380e-16,8.054150e-15,5.256250e-15,1.387430e-14,5.335940e-15,4.120870e-15,7.106870e-16,-0.093199,0.130023,-0.270600,0.174687,-0.298847,0.230794,0.511470,0.244355,0.0,NaN,-0.085054,NaN,NaN,NaN,NaN,0,1.287610e-14,6.683010e-15,1.562970e-14,8.862460e-15,54750.880949,54781.372685,2,False,http://xmm-catalog.irap.omp.eu/source/20554120...,NaN,1.0,0.237952,2.195549e-15,4.146526e-16,0.441303
428,SRGe J104810.7+570655,162.044779,57.115203,8.602118e-15,2.146331,6.003609,45.693610,0.0,6.603970,205562115010017,4XMM J104810.9+570656,3.171050,1.133672,162.045523,57.115656,1.598640,24.7796,3.989960e-15,1.711790e-15,5.474000e-15,1.885930e-15,7.698520e-15,2.681680e-15,7.419620e-15,4.961540e-15,6.603190e-15,2.118000e-14,2.678970e-14,2.228610e-14,2.439260e-14,5.495440e-15,-0.000612,0.246883,0.266634,0.211565,-0.376687,0.256557,0.060874,0.525925,0.0,NaN,-0.847301,NaN,NaN,NaN,NaN,0,2.678970e-14,2.228610e-14,2.678970e-14,2.228610e-14,54791.123333,54791.204734,1,False,h

In [11]:
xmm_desi = cross_match_data_frames(ero_xmm, desi, colname_ra1 = 'SC_RA',colname_dec1 = 'SC_DEC' , colname_ra2 = 'ra', colname_dec2=  'dec', match_radius = 15, df_prefix = 'desi')

cross-match radius 15 arcsec
total matches: 3284 out of 740 x 2418574
	 total unique pairs: 3284
	 total non-unique pairs (duplicates in df2): 0


In [12]:
xmm_desi_closest = (xmm_desi
    .loc[xmm_desi.groupby('IAUNAME')['desi_sep'].idxmin()]
    )


xmm_hostless = (xmm_desi_closest[
    xmm_desi_closest['desi_sep'] > 2 * xmm_desi_closest['xmm_pos_r98']
    ])
xmm_hostless = xmm_hostless.query('Separation<20')

print('hostless xmm near chandra sources', xmm_hostless.shape[0])
xmm_hostless = xmm_hostless[['srcname_fin',	'RA_fin',	'DEC_fin', 'pos_r98', 'flux_05-20', 'IAUNAME', 'SC_RA',	'SC_DEC', 'xmm_pos_r98', 'flux_05_2', 'Separation', 'desi_desi_id', 'desi_ra', 'desi_dec', 'desi_sep', 'xmm_ero_flux_ratio']]

xmm_hostless.rename(columns = {'Separation':'ero_xmm_sep', 'desi_sep':'desi_sep_minimal'}, inplace = True)

xmm_hostless = add_separation_columns(xmm_hostless, 'RA_fin', 'DEC_fin', 'desi_ra', 'desi_dec', 'desi_ero_sep')

assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.IAUNAME.nunique()
assert xmm_hostless.srcname_fin.nunique()==xmm_hostless.shape[0]
xmm_hostless.sample(7)

hostless xmm near chandra sources 38


,srcname_fin,RA_fin,DEC_fin,pos_r98,flux_05-20,IAUNAME,SC_RA,SC_DEC,xmm_pos_r98,flux_05_2,ero_xmm_sep,desi_desi_id,desi_ra,desi_dec,desi_sep_minimal,xmm_ero_flux_ratio,desi_ero_sep
144,SRGe J104114.5+590218,160.310425,59.038426,3.082154,4.098946e-14,4XMM J104114.4+590219,160.310300,59.038744,1.061793,5.099540e-14,1.165737,9011_614514_712,160.309144,59.038730,2.141124,1.244110,2.611861
3132,SRGe J105008.1+572515,162.533608,57.420888,10.038040,2.905046e-15,4XMM J105008.3+572513,162.534868,57.420442,1.507413,2.722390e-15,2.922590,9011_609949_916,162.531100,57.418207,10.865947,0.937125,10.806199
1483,SRGe J104831.3+584027,162.130517,58.674177,5.318124,8.632975e-15,4XMM J104830.9+584024,162.129072,58.673516,2.449915,5.889160e-15,3.599889,9011_613769_2926,162.130705,58.674620,5.011373,0.682170,1.633069
2878,SRGe J104952.5+571919,162.468576,57.322030,8.281549,3.364370e-15,4XMM J104952.8+571925,162.470002,57.323768,3.318609,2.541130e-15,6.844347,9011_609168_3700,162.471935,57.326130,9.295602,0.755306,16.139947
2067,SRGe J104548.9+590827,161.453772,59.140960,6.432184,5.889830e-15,4XMM J104549.0+590830,161.454383,59.141691,1.381511,4.509600e-15,2.862935,9011_615259_3071,161.452012,59.140319,6.602473,0.765659,3.988206
2153,SRGe J103349.3+584441,158.455573,58.744828,6.972946,5.515691e-15,4XMM J103349.2+584440,158.455253,58.744687,3.907439,9.506640e-15,0.782359,9011_613762_405,158.448776,58.745133,12.203850,1.723563,12.743523
3041,SRGe J105155.8+572500,162.982692,57.416461,11.389185,3.084923e-15,4XMM J105155.2+572458,162.980135,57.416149,0.899890,1.456735e-15,5.083507,9011_609950_914,162.984824,57.414891,10.157926,0.472211,7.002669


# Joining CSC and XMM hostless candidates

In [13]:
csc_ctps = csc_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'pos_r98', 'flux_05-20', 'ero_csc_sep', 'csc_name',   'csc_ra', 'csc_dec', 'csc_flux_05_2', 'csc_r_98_csc', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'desi_ero_sep', 'csc_ero_flux_ratio']]

xmm_ctps = xmm_hostless[['srcname_fin', 'RA_fin','DEC_fin', 'pos_r98', 'flux_05-20', 'ero_xmm_sep','IAUNAME', 'SC_RA','SC_DEC',  'flux_05_2', 'xmm_pos_r98', 'desi_sep_minimal', 'desi_desi_id',  'desi_ra', 'desi_dec', 'desi_ero_sep',  'xmm_ero_flux_ratio']]

csc_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)
xmm_ctps.rename(columns = {'desi_desi_id':'desi_id'}, inplace = True)

In [14]:
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] <  5 ]
xmm_ctps = xmm_ctps[xmm_ctps['xmm_ero_flux_ratio'] >  1/5 ]

csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] <  5 ]
csc_ctps = csc_ctps[csc_ctps['csc_ero_flux_ratio'] >  1/5 ]

In [15]:
xmm_ctps['x_ray_det'] = 'xmm'
csc_ctps['x_ray_det'] = 'csc'
print('XMM companions', xmm_ctps.shape[0])
print('CSC companions', csc_ctps.shape[0])
print('Total companions possible', xmm_ctps.shape[0] + csc_ctps.shape[0])

XMM companions 37
CSC companions 21
Total companions possible 58


In [16]:
final_ctps = pd.concat([xmm_ctps, csc_ctps])
final_ctps.sort_values(by='srcname_fin', inplace=True)
final_ctps.reset_index(drop=True, inplace=True)


n_ctps = final_ctps.groupby('srcname_fin')['x_ray_det'].transform(lambda x: len(x.unique())) 
final_ctps['n_x_ray_det'] = n_ctps


n_desi_ctps = final_ctps.groupby('srcname_fin')['desi_id'].transform(lambda x: len(x.unique()))
final_ctps['n_desi_ctps'] = n_desi_ctps


final_ctps = final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )')
## final_ctps.query('( n_x_ray_det == 1 ) | ( n_x_ray_det==2 & n_desi_ctps == 1 )') - opposite of this


final_ctps.drop_duplicates(subset=['srcname_fin', 'desi_id'], inplace=True)
print('Final number of hostless', final_ctps.shape[0])
final_ctps.reset_index()



Final number of hostless 54


,index,srcname_fin,RA_fin,DEC_fin,pos_r98,flux_05-20,ero_xmm_sep,IAUNAME,SC_RA,SC_DEC,flux_05_2,xmm_pos_r98,desi_sep_minimal,desi_id,desi_ra,desi_dec,desi_ero_sep,xmm_ero_flux_ratio,x_ray_det,ero_csc_sep,csc_name,csc_ra,csc_dec,csc_flux_05_2,csc_r_98_csc,csc_ero_flux_ratio,n_x_ray_det,n_desi_ctps
0,0,SRGe J103121.9+573134,157.841091,57.526169,8.679921,4.350933e-15,NaN,NaN,NaN,NaN,NaN,NaN,8.192553,9011_609939_482,157.839339,57.524106,8.162275,NaN,csc,0.504317,2CXO J103121.9+573134,157.841331,57.526115,3.862979e-15,2.331750,0.887851,1,1
1,1,SRGe J103158.3+573841,157.993098,57.644831,11.907729,3.019065e-15,NaN,NaN,NaN,NaN,NaN,NaN,8.059029,9011_610714_3345,157.981929,57.647060,22.964569,NaN,csc,14.930985,2CXO J103156.5+573846,157.985747,57.646145,1.189964e-14,1.155268,3.941499,1,1
2,2,SRGe J103220.4+573211,158.084926,57.536450,5.714911,9.351694e-15,NaN,NaN,NaN,NaN,NaN,NaN,9.735829,9011_609939_2890,158.088107,57.538123,8.606599,NaN,csc,1.421180,2CXO J103220.2+573211,158.084192,57.536421,4.852860e-15,1.498974,0.518928,1,1
3,3,SRGe J103239.4+574033,158.163965,57.675913,6.556224,8.782485e-15,3.169250,4XMM J103239.1+574036,158.163236,57.676703,5.205230e-15,1.764312,11.353960,9011_610715_599,158.169132,57.676603,10.251895,0.592683,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
4,5,SRGe J103251.8+574550,158.215643,57.763855,11.176683,2.601944e-15,4.011813,4XMM J103252.0+574546,158.216941,57.762982,2.699190e-15,1.462532,8.608715,9011_610715_1014,158.221411,57.763161,11.355228,1.037374,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,1
5,7,SRGe J103302.4+580241,158.260049,58.044662,6.663467,6.085609e-15,NaN,NaN,NaN,NaN,NaN,NaN,6.516825,9011_611484_2685,158.264079,58.045970,9.006889,NaN,csc,3.061539,2CXO J103302.7+580240,158.261655,58.044693,5.264955e-15,2.090497,0.865148,1,1
6,8,SRGe J103349.3+584441,158.455573,58.744828,6.972946,5.515691e-15,0.782359,4XMM J103349.2+584440,158.455253,58.744687,9.506640e-15,3.907439,12.203850,9011_613762_405,158.448776,58.745133,12.743523,1.723563,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
7,9,SRGe J103409.4+574725,158.539363,57.790228,7.180546,4.495352e-15,NaN,NaN,NaN,NaN,NaN,NaN,6.808410,9011_610715_3565,158.543036,57.792069,9.673252,NaN,csc,3.623997,2CXO J103409.5+574728,158.539889,57.791195,2.186806e-15,1.230499,0.486459,1,1
8,10,SRGe J103518.6+562405,158.827436,56.401399,6.233959,8.711545e-15,5.638750,4XMM J103518.0+562408,158.825121,56.402300,9.770200e-15,3.030869,6.995479,9011_606788_302,158.827919,56.401126,1.375083,1.121523,xmm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1
9,11,SRGe J103520.2+573355,158.834253,57.565203,8.727764,3.591921e-15,NaN,NaN,NaN,NaN,NaN,NaN,6.015550,9011_609941_914,158.835675,57.562204,11.139738,NaN,csc,7.821427,2CXO J103520.9+573349,158.837140,57.563679,4.241985e-15,2.384348,1.180980,1,1


In [17]:
# #save final_ctps_csv to csv
final_ctps_csv_coords = final_ctps[['srcname_fin', 'RA_fin',	'DEC_fin',	'pos_r98']]
final_ctps_csv_coords_matched = cross_match_data_frames(final_ctps_csv_coords, desi[['desi_id', 'ra', 'dec', 'type']], 'RA_fin', 'DEC_fin', 'ra', 'dec', closest=True, match_radius=30)
final_ctps_csv_coords_matched

cross-match radius 30 arcsec
total matches: 684 out of 54 x 2418574
	 total unique pairs: 684
	 total non-unique pairs (duplicates in df2): 0
total closest matches: 54


,srcname_fin,RA_fin,DEC_fin,pos_r98,desi_id,ra,dec,type,sep,n_near,n_matches
12,SRGe J103121.9+573134,157.841091,57.526169,8.679921,9011_609939_482,157.839339,57.524106,REX,8.162275,17,1
25,SRGe J103158.3+573841,157.993098,57.644831,11.907729,9011_610714_3400,157.988800,57.643752,PSF,9.144548,12,1
30,SRGe J103220.4+573211,158.084926,57.536450,5.714911,9011_609939_2890,158.088107,57.538123,PSF,8.606599,12,1
43,SRGe J103239.4+574033,158.163965,57.675913,6.556224,9011_610715_599,158.169132,57.676603,DEV,10.251895,16,1
61,SRGe J103251.8+574550,158.215643,57.763855,11.176683,9011_610715_1014,158.221411,57.763161,REX,11.355228,12,1
72,SRGe J103302.4+580241,158.260049,58.044662,6.663467,9011_611484_2685,158.264079,58.045970,REX,9.006889,5,1
81,SRGe J103349.3+584441,158.455573,58.744828,6.972946,9011_613762_405,158.448776,58.745133,PSF,12.743523,11,1
97,SRGe J103409.4+574725,158.539363,57.790228,7.180546,9011_610715_3500,158.536213,57.788361,REX,9.038309,15,1
105,SRGe J103518.6+562405,158.827436,56.401399,6.233959,9011_606788_302,158.827919,56.401126,REX,1.375083,18,1
121,SRGe J103520.2+573355,158.834253,57.565203,8.727764,9011_609941_904,158.834278,57.566603,REX,5.041880,15,1


In [18]:
final_ctps_csv_coords_matched.query('sep>pos_r98 & type=="PSF"')

,srcname_fin,RA_fin,DEC_fin,pos_r98,desi_id,ra,dec,type,sep,n_near,n_matches
30,SRGe J103220.4+573211,158.084926,57.536450,5.714911,9011_609939_2890,158.088107,57.538123,PSF,8.606599,12,1
81,SRGe J103349.3+584441,158.455573,58.744828,6.972946,9011_613762_405,158.448776,58.745133,PSF,12.743523,11,1
137,SRGe J103525.8+595331,158.857628,59.891915,8.250369,9011_617451_1827,158.861910,59.888438,PSF,14.712714,8,1
165,SRGe J103748.7+585641,159.452829,58.944819,5.360453,9011_614512_1573,159.448476,58.943895,PSF,8.741244,22,1
243,SRGe J104122.8+590252,160.344824,59.047725,8.902710,9011_614514_927,160.339396,59.048743,PSF,10.696558,15,1
280,SRGe J104341.0+590023,160.920944,59.006447,10.333953,9011_614515_1534,160.919953,59.009420,PSF,10.858004,14,1
301,SRGe J104453.1+585450,161.221287,58.913756,8.352394,9011_614516_382,161.226910,58.914631,PSF,10.915065,8,1
395,SRGe J104656.6+572859,161.735777,57.482989,5.116249,9011_609947_2233,161.739150,57.482406,PSF,6.854900,12,1
662,SRGe J105640.4+573203,164.168282,57.534091,6.063802,9011_609952_2647,164.167686,57.536689,PSF,9.422389,14,1


In [19]:
# #save final_ctps_csv to csv
final_ctps_csv = final_ctps[['srcname_fin', 'desi_id']]
final_ctps_csv.rename(columns={'srcname_fin':'ID'}, inplace=True)
final_ctps_csv.rename(columns={'desi_id':'desi_id_true_ctp'}, inplace=True)
final_ctps_csv.desi_id_true_ctp = 'hostless'
final_ctps_csv.ID = final_ctps_csv.ID.str.encode('utf-8')
final_ctps_csv.desi_id_true_ctp = final_ctps_csv.desi_id_true_ctp.str.encode('utf-8')
final_ctps_csv.to_pickle(data_path+'validation_ctps_ero_desi_lh_hostless.pkl')
final_ctps_csv

,ID,desi_id_true_ctp
0,b'SRGe J103121.9+573134',b'hostless'
1,b'SRGe J103158.3+573841',b'hostless'
2,b'SRGe J103220.4+573211',b'hostless'
3,b'SRGe J103239.4+574033',b'hostless'
5,b'SRGe J103251.8+574550',b'hostless'
7,b'SRGe J103302.4+580241',b'hostless'
8,b'SRGe J103349.3+584441',b'hostless'
9,b'SRGe J103409.4+574725',b'hostless'
10,b'SRGe J103518.6+562405',b'hostless'
11,b'SRGe J103520.2+573355',b'hostless'


# Combine non-hostless and hostless catalogs to produce final validation sample

In [20]:
df_ctps = pd.read_pickle(data_path+'validation_ctps_ero_desi_lh_no_hostless.pkl')
df_hostless = pd.read_pickle(data_path+'validation_ctps_ero_desi_lh_hostless.pkl')
df_ctps = pd.concat([df_ctps, df_hostless])

df_ctps.to_pickle(data_path+'validation_ctps_ero_desi_lh.pkl')
df_ctps.sample(10)

,ID,desi_id_true_ctp
454,b'SRGe J104859.8+565647',b'9011_608383_455'
369,b'SRGe J104630.4+584435',b'9011_613768_2422'
671,b'SRGe J110542.4+590043',b'9011_614526_3153'
67,b'SRGe J103341.6+573644',b'9011_609940_1371'
331,b'SRGe J104526.4+560201',b'9011_605186_688'
19,b'SRGe J103222.2+584058',b'9011_613761_1548'
356,b'SRGe J104616.0+585014',b'9011_613768_1990'
453,b'SRGe J104859.2+583130',b'9011_613016_2815'
309,b'SRGe J104458.5+555749',b'9011_605185_3710'
230,b'SRGe J104140.8+593807',b'9011_616727_1259'
